In [3]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Tell Dash to look for assets one level up
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)

app.title = "🎀 Decoration Carousel"

# 📦 Sous-catégories dans assets/object/decoration/
DECORATION_PATH = os.path.abspath("../assets/object/decoration")
DECORATION_ITEMS = sorted([
    item for item in os.listdir(DECORATION_PATH)
    if os.path.isdir(os.path.join(DECORATION_PATH, item))
])

# 📦 Dictionnaire {item: [liste d’images]}
decoration_images = {
    item: sorted([
        os.path.join(DECORATION_PATH, item, img)
        for img in os.listdir(os.path.join(DECORATION_PATH, item))
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for item in DECORATION_ITEMS
}

# 🎨 Layout principal
app.layout = html.Div([
    html.H2("\ud83c\udf80 Decoration Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select decoration item:", style={
            "fontSize": "16px", "marginBottom": "6px", "fontFamily": "Georgia, serif"
        }),
        dcc.Dropdown(
            id='item-dropdown',
            options=[{
                "label": item.replace("_", " ").capitalize(),
                "value": item
            } for item in DECORATION_ITEMS],
            value=DECORATION_ITEMS[0],
            clearable=False,
            style={"width": "300px", "margin": "0 auto", "fontFamily": "Georgia, serif"}
        )
    ], style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
], style={"fontFamily": "Georgia, serif", "padding": "30px"})

# 🔄 Afficher le bon carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value')
)
def update_carousel(item):
    images = decoration_images.get(item, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))

    return html.Div([
        html.Div([
            html.Button("\u2b05\ufe0f", id="prev-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginRight": "15px"}),
            html.Img(id="carousel-image", src=f"/assets/{relative_path}",
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("\u27a1\ufe0f", id="next-button", n_clicks=0,
                        style={"fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px"}),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),

        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontFamily": "Georgia, serif", "color": "#444"})
    ])

# 🔄 Navigation flèches
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Lancer (changer le port si besoin)
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8204)
